## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
    mean = np.mean(X_train,axis=(0,1,2,3))
    std = np.std(X_train, axis=(0, 1, 2, 3))
    X_train = (X_train-mean)/(std+1e-7)
    X_test = (X_test-mean)/(std+1e-7) 
    return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

170500096/170498071 [==============================] - 4s 0us/step
(50000, 32, 32, 3)


In [0]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [4]:

classifier=Sequential()

#卷積組合
#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(Convolution2D(filters=32, kernel_size=(3, 3), input_shape=(32, 32, 3), activation='relu'))
classifier.add(BatchNormalization())



#卷積組合
classifier.add(Convolution2D(32, kernel_size=(3, 3), activation="relu"))
classifier.add(BatchNormalization(momentum=0.98, epsilon=0.001))
# '''自己決定MaxPooling2D放在哪裡''' ----> 放這    
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(units=100, activation="relu")) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.summary()
classifier.fit(x_train,y_train,batch_size=100,epochs=100)















Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 28, 28, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`




Epoch 1/100
50000/50000 [==============================] - 11s 220us/step - loss: 1.3710 - acc: 0.5204
Epoch 2/100
50000/50000 [==============================] - 4s 88us/step - loss: 0.9437 - acc: 0.6712
Epoch 3/100
50000/50000 [==============================] - 4s 87us/step - loss: 0.7613 - acc: 0.7323
Epoch 4/100
50000/50000 [==============================] - 4s 87us/step - loss: 0.6190 - acc: 0.7814
Epoch 5/100
50000/50000 [==============================] - 4s 87us/step - loss: 0.4933 - acc: 0.8257
Epoch 6/100
50000/50000 [==============================] - 4s 88us/step - loss: 0.3636 - acc: 0.8766
Epoch 7/100
50000/50000 [==============================] - 4s 87us/step - loss: 0.2627 - acc: 0.9090
Epoch 8/100
50000/50000 [==============================] - 4s 87us/step - loss: 0.1967 - acc: 0.9324
Epoch 9/100
50000/50000 [==============================] - 4s 89us/step - loss: 0.1529 - acc: 0.9487
Epoch 10/100
50000/50000 [==============================] - 4s 89us/step - loss: 0.1238

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[1.6524389e-09, 0.0000000e+00, 3.3007255e-18, 9.9803597e-01,
        1.9641006e-03, 0.0000000e+00, 1.5776334e-18, 0.0000000e+00,
        9.1417173e-28, 0.0000000e+00]], dtype=float32)